# Julia Interactive Example: TinyMPC on Cartpole (Extended Version)
In this demonstration, we showcase an interactive Julia workflow using TinyMPC. You can generate C++ code and engage with it seamlessly within the Julia environment. This example guides you through the entire workflow, beginning with the cartpole's nonlinear dynamics.

If any issues arise, restart the kernel.

Load necessary packages, make sure to install `tinympc` ([README.md](../README.md))

In [ ]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

using Libdl
using LinearAlgebra
import ForwardDiff as FD

include(joinpath(@__DIR__,"../tinympc/TinyMPC.jl"))
using .TinyMPC
include("visualization.jl") # for visualization

## Cartpole Dynamics
Build the cartpole nonlinear dynamics and linearize around upright state using `ForwardDiff`. 

In [ ]:
freq = 100  # frequency of the controller and dynamics Hz
dt = 1/freq  # time step

# cartpole dynamics with theta = 0 at the down position
function cartpole_dynamics(x::Vector, u::Vector)
    g = -9.8  # gravity m/s^2
    m = 0.2  # mass of the pole kg
    M = 0.5  # mass of the cart kg
    l = 0.3  # length of the pole m

    x, x_dot, theta, theta_dot = x
    u = u[1]

    x_ddot = (u + m * l * theta_dot^2 * sin(theta) - m * g * sin(theta) * cos(theta)) / (M + m * sin(theta)^2)
    theta_ddot = (-u * cos(theta) - m * l * theta_dot^2 * sin(theta) * cos(theta) + (M + m) * g * sin(theta)) / (l * (M + m * sin(theta)^2))

    return [x_dot, x_ddot, theta_dot, theta_ddot]
end    
function rk4(x::Vector, u::Vector)
    f = cartpole_dynamics
    k1 = f(x, u)
    k2 = f(x + dt * k1 / 2, u)
    k3 = f(x + dt * k2 / 2, u)
    k4 = f(x + dt * k3, u)
    return x + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
end
function cartpole_dynamics_rk4(x::Vector, u::Vector)
    return rk4(x, u)
end

# Linearize the dynamics around x0, u0
x0 = [0, 0, pi, 0.0]
u0 = [0.]
Anp = FD.jacobian(x -> cartpole_dynamics_rk4(x, u0), x0)
Bnp = FD.jacobian(u -> cartpole_dynamics_rk4(x0, u), u0)
display(Anp)
display(Bnp)

In [ ]:
x_all = [zeros(4) for i in 1:300]
# simulate the dynamics with the zero controller
x = [0, 0, 0.1, 0]
for i in 1:300
    x = cartpole_dynamics_rk4(x, u0)
    x_all[i] = x
end
# Go to the visualization part at the end and run it to see the trajectory `x_all`

## LQR Controller

Let's run LQR on the linearized model

In [ ]:
# Riccati recursion on the linearized dynamics
Q = Diagonal([10, 1, 10, 1.])
R = Diagonal([1.])
P = 1*Q
K = zeros(1, 4)
for i in 1:100
    P = Q + Anp' * P * Anp - Anp' * P * Bnp * inv(R + Bnp' * P * Bnp) * Bnp' * P * Anp
    K = inv(R + Bnp' * P * Bnp) * Bnp' * P * Anp
end
display(K)

# LQR controller
function lqr_controller(x::Vector)
    return -K * x
end

# simulate the dynamics with the LQR controller
x = [0, 0, pi - 0.1, 0.0] # initial state
for i in 1:300
    u = lqr_controller(x - x0)
    x = cartpole_dynamics_rk4(x, u)
    x_all[i] = x
end
# Go to the visualization part at the end and run it to see the trajectory `x_all`

## Code Generation

We are done with the dynamics and LQR controller. Now, let's define the class and compile original TinyMPC code to get a generic shared/dynamic library

**PLEASE CHANGE `tinympc_julia_dir` TO YOUR ABSOLUTE PATH**

In [ ]:
tinympc_julia_dir = "/home/khai/SSD/Code/tinympc-julia"  # Your absolute path to the tinympc-Julia directory, you need to change this
tinympc_dir = tinympc_julia_dir * "/tinympc/TinyMPC"  # Path to the TinyMPC directory (C code)
TinyMPC.compile_lib(tinympc_dir)  # Compile the C code into a shared library

Load the generic shared/dynamic library. **You may want to change the extension of the library based on your OS -- Linux: .so, Mac: .dylib, Windows: .dll**

In [ ]:
os_ext = ".so"  # CHANGE THIS BASED ON YOUR OS
tinympc = tinympc_dir * "/build/src/tinympc/libtinympcShared" * os_ext  # Path to the compiled library

Here we setup problem data and settings for TinyMPC

In [ ]:
n = 4
m = 1
N = 10

# convert A to array column major
# A = Array{Float32}(cat(Anp..., dims=2))[:]
A = cat(Anp..., dims=2)[:]
B = cat(Bnp..., dims=2)[:]
display(A')
display(B')
Q = [10.0, 1, 10, 1]
R = [1.0]
rho = 0.1

x_min = -5. * ones(n*N)  # state constraints
x_max = 5. * ones(n*N)  # state constraints
u_min = -5 * ones(m*(N-1))  # force constraints
u_max = 5 * ones(m*(N-1))  # force constraints

abs_pri_tol = 1.0e-3  # absolute primal tolerance
abs_dual_tol = 1.0e-3  # absolute dual tolerance
max_iter = 100  # maximum number of iterations
check_termination = 1  # whether to check termination and period

After define the problem, we generate the tailored code with above data. 

**Here we compile it for interactive Julia script but you can use it directly for your applications/systems**

In [ ]:
output_dir = tinympc_julia_dir * "/generated_code1"  # Path to the generated code

@ccall tinympc.tiny_codegen(n::Cint, m::Cint, N::Cint, A::Ptr{Float64}, B::Ptr{Float64}, Q::Ptr{Float64}, R::Ptr{Float64}, x_min::Ptr{Float64}, x_max::Ptr{Float64}, u_min::Ptr{Float64}, u_max::Ptr{Float64}, rho::Float64, abs_pri_tol::Float64, abs_dual_tol::Float64, max_iter::Cint, check_termination::Cint, 1::Cint, tinympc_dir::Ptr{UInt8}, output_dir::Ptr{UInt8})::Cint

TinyMPC.compile_lib(output_dir)

## Interactive MPC

Run the interactive MPC example which calls the generated code, use nonlinear dynamics for simulation

Since this works with pointers, underlying data is persistent in each kernel session (something like deepcopy/shallowcopy or pass by reference/value). If you want to run from the original setup, you may need to change data back or 
restart kernel.

In [1]:
NSIM = 400
x_all = [zeros(n) for i in 1:NSIM]

tinympc = output_dir * "/build/tinympc/libtinympcShared" * os_ext  # Path to the compiled library

x = [0.5, 0, -0.4 + pi, 0.1] # Initial state
u = Array{Float32}(zeros(m*(N-1)))  # List of control inputs in horizon

delta_x_noise = Array{Float32}(x - x0)

# Use delta because MPC uses the linearized dynamics around upright position
# Set the reference state to 0 as well as reset
delta_xref = Array{Float32}(zeros(n*N))  # reference state
@ccall tinympc.set_xref(delta_xref::Ptr{Float32}, 0::Cint)::Cvoid

# Set the reference x to 1 at step 200
delta_xref_new_ = [[1.0, 0, 0, 0] for i in 1:N]
delta_xref_new = Array{Float32}(cat(delta_xref_new_..., dims=2))[:]


for i in 1:NSIM
    # 1. Set initial state from measurement   
    @ccall tinympc.set_x0(delta_x_noise::Ptr{Float32}, 0::Cint)::Cvoid

    # 2. Set the reference state if needed
    # At step 200, set x = 1
    if (i==200)
        @ccall tinympc.set_xref(delta_xref_new::Ptr{Float32}, 0::Cint)::Cvoid
    end

    # 3. Solve the problem
    @ccall tinympc.call_tiny_solve(0::Cint)::Cvoid

    # 4. Get the control input
    @ccall tinympc.get_u(u::Ptr{Float32}, 0::Cint)::Cvoid

    # 5. Simulate the dynamics
    x = cartpole_dynamics_rk4(x, u)

    noise = randn(n) * 0.01
    delta_x_noise = Array{Float32}(x + noise - x0)
    x_all[i] = x
end


UndefVarError: UndefVarError: n not defined

In [ ]:
# Visualize the result (may take some time)
display(animate_cartpole(x_all, dt))

## Deployment

Post testing the MPC procedure with the generated code, the next step involves deploying it for your specific applications/systems.
The workflow for deployment is tailored to your specific needs, and we aim to provide clear guidance.

Your `tiny_main` may look like this

```C
int main()
{
    int exitflag = 1;
    TinyWorkspace* work = tiny_data_solver.work;
    tiny_data_solver.work->Xref = tiny_MatrixNxNh::Zero();
    tiny_data_solver.work->Uref = tiny_MatrixNuNhm1::Zero();
    tiny_data_solver.settings->max_iter = 150;
    tiny_data_solver.settings->en_input_bound = 1;
    tiny_data_solver.settings->en_state_bound = 1;

    tiny_VectorNx x0, x1; // current and next simulation states
    x0 << 0.0, 0, 0.1, 0; // initial state

    int i = 0;
    for (int k = 0; k < 300; ++k)
    {
        printf("tracking error at step %2d: %.4f\n", k, (x0 - work->Xref.col(1)).norm());

        // 1. Update measurement
        work->x.col(0) = x0;

        // 2. Update reference (if needed)
        // you can also use C wrapper (intended for high-level languages) 
        // by including tiny_wrapper.hpp and call `set_xref(...)` function

        // 3. Reset dual variables (if needed)
        work->y = tiny_MatrixNuNhm1::Zero();
        work->g = tiny_MatrixNxNh::Zero();

        // 4. Solve MPC problem
        exitflag = tiny_solve(&tiny_data_solver);

        // if (exitflag == 0)
        // 	printf("HOORAY! Solved with no error!\n");
        // else
        // 	printf("OOPS! Something went wrong!\n");
        // 	// break;

        std::cout << work->iter << std::endl;
        std::cout << work->u.col(0).transpose().format(CleanFmt) << std::endl;

        // 5. Simulate forward
        // work->u.col(0) = -tiny_data_solver.cache->Kinf * (x0 - work->Xref.col(0));
        x1 = work->Adyn * x0 + work->Bdyn * work->u.col(0);
        x0 = x1;
        // std::cout << x0.transpose().format(CleanFmt) << std::endl;
    }
}
```